In [8]:
import pdfplumber
import re
import sys
import os
import json

In [9]:
if (sys.platform == 'linux'):
    pdf = pdfplumber.open("/home/adamson/projects/cs/ref/angus/16_Itikawa_J_Phys_Chem_Ref_Data_35_31_2006.pdf")

In [10]:
cropDim=[0 for i in range(len(pdf.pages))]

In [29]:
"_".join(['process','lhs','rhs'])

'process_lhs_rhs'

In [106]:
def p4FilenameGenerator():
    return "../../data/raw/ext/n2/itikawa/total"

In [12]:
def p5FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/elastic"

In [13]:
def p6FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/momentum"

In [14]:
def p7FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/rot_0_2"

In [15]:
def p8FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/vib_0_1"

In [16]:
def p10FilenameGenerator(i):
    return "../../data/raw/ext/itikawa_n2/tr_001_" + str(i + 2).zfill(3)

In [17]:
def p11FilenameGenerator(i):
    return "../../data/raw/ext/itikawa_n2/tr_001_" + str(i + 6).zfill(3)

In [18]:
def itikawa_table(ipage, filenameGenerator, fileHeader, locateTables=False):
    if (cropDim[ipage] != 0):
        pg = pdf.pages[ipage]
        for i in range(len(cropDim[ipage])):
            pgCropped = pg.crop(cropDim[ipage][i])
            if (locateTables):
                display(pgCropped.to_image().reset().draw_rects(pgCropped.chars))
            else:
                textSpaceDelim = pgCropped.extract_text().split("\n")
                textCommaDelim = fileHeader + "\n"
                for j in range(len(textSpaceDelim)):
                    textCommaDelim = textCommaDelim + (re.sub("\s+", ",", textSpaceDelim[j].strip())) + "\n"
                filename = filenameGenerator(i)
                exists = os.path.isfile(filename)
                if exists:
                    print("Skipping " + filename + ". It exists already.")
                else:
                    f = open(filename, "x")
                    f.write(textCommaDelim)
                    f.close()

In [101]:
#TODO: refactor the multiple calls to display(pgCropped...)
def itikawa_multicolumn_table(ipage, filenameGenerator, fileHeader, locateTables=False, omitRegexp=''):
    if (cropDim[ipage] != 0):
        pg = pdf.pages[ipage]
        filename = filenameGenerator()
        dataFilename = filename + "_data"
        exists = os.path.isfile(filename)
        existsData = os.path.isfile(dataFilename)
        if (exists or existsData):
            print("Skipping " + filename + " and " + dataFilename + ". At least one exists already.")
        else:
            f = open(filename, "x")
            f.write(fileHeader)
            f.close()
            if (locateTables==False):
                fdata = open(dataFilename, "x")
                textTabDelim = ""
                for i in range(len(cropDim[ipage])):
                    pgCropped = pg.crop(cropDim[ipage][i])
                    textSpaceDelim = pgCropped.extract_text().split("\n")
                    for j in range(len(textSpaceDelim)):
                        if (omitRegexp == '' or not(bool(re.search(omitRegexp, textSpaceDelim[j])))):
                            textTabDelim = textTabDelim + ("\t" + re.sub("\s+", "\t", textSpaceDelim[j].strip())) + "\n"
                fdata.write(textTabDelim)
                fdata.close()
            else:
                for i in range(len(cropDim[ipage])):
                    pgCropped = pg.crop(cropDim[ipage][i])
                    display(pgCropped.to_image().reset().draw_rects(pgCropped.chars))


In [96]:
#TODO: refactor the multiple calls to display(pgCropped...)
def itikawa_multicolumn_table_json(ipage, 
                                   filename,
                                   process, 
                                   units, 
                                   specie, 
                                   lhs, rhs='', locateTables=False, omitRegexp='', comment=''):
    if (cropDim[ipage] != 0):
        pg = pdf.pages[ipage]
        energies=[]
        sigmas=[]
        exists = os.path.isfile(filename)
        if exists:
            print("Skipping " + filename + ". It exists already.")
        else:
            if (locateTables==False):
                f = open(filename, "x")
                for i in range(len(cropDim[ipage])):
                    pgCropped = pg.crop(cropDim[ipage][i])
                    textSpaceDelim = pgCropped.extract_text().split("\n")
                    for j in range(len(textSpaceDelim)):
                        if (omitRegexp == '' or not(bool(re.search(omitRegexp, textSpaceDelim[j])))):
                            textArray = textSpaceDelim[j].split(' ')
                            energies.append(textArray[0])
                            sigmas.append(textArray[1])
                textToWrite = "\t".join(['', specie, lhs, rhs, process, json.dumps({'units':units,'E':energies,'sigma':sigmas,'comment':comment})])
                f.write(textToWrite)
                f.close()
                #print(json.dumps([{'units':units,'E':energies,'sigma':sigmas,'comment':comment}]))
            else:
                for i in range(len(cropDim[ipage])):
                    pgCropped = pg.crop(cropDim[ipage][i])
                    display(pgCropped.to_image().reset().draw_rects(pgCropped.chars))


In [97]:
cropDim[3] = [(300, 503, 390, 720),(400, 503, 470, 720),(480, 503, 560, 720)]

In [99]:
itikawa_multicolumn_table_json(3, filename='../../data/raw/ext/n2/itikawa/total_N2', 
                               process='total',
                               units={'E':'eV','sigma':'1E-16 cm^2'}, 
                               specie="N2",
                               lhs='',
                               rhs='',
                               locateTables=False, 
                               omitRegexp='[0-9]a',
                               comment='this is a comment')

In [102]:
cropDim[3] = [(300, 503, 390, 720),(400, 503, 470, 720),(480, 503, 560, 720)]

In [108]:
itikawa_multicolumn_table(3, p4FilenameGenerator, "\tN2\t\ttotal", locateTables=False, omitRegexp='[0-9]a')

In [110]:
cropDim[4] = [(300, 470, 420, 730),(430, 470, 550, 730)]

In [115]:
itikawa_multicolumn_table(4, p5FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [139]:
cropDim[5] = [(300, 105, 390, 333),(390, 105, 480, 333),(480, 105, 570, 333)]

In [141]:
itikawa_multicolumn_table(5, p6FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [153]:
cropDim[6] = [(50, 105, 170, 275),(180, 105, 300, 275)]

In [155]:
itikawa_multicolumn_table(6, p7FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False)

In [172]:
cropDim[7] = [(52, 541, 300, 721)]

In [174]:
itikawa_multicolumn_table(7, p8FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [23]:
cropDim[9] = [(120, 440, 200, 750),(210,440,290,750),(290,440,400,750),(400,440,500,750)]

In [58]:
itikawa_table(9, p10FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False)

Skipping ../../data/raw/ext/itikawa_n2/tr_001_002. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_003. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_004. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_005. It exists already.


In [59]:
cropDim[10] = [(40, 120, 120, 450),(130,120,210,450),(210,120,290,450),
               (310,120,400,450),(400,120,480,450),(480,120,560,450)]

In [60]:
itikawa_table(10, p11FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)")

Skipping ../../data/raw/ext/itikawa_n2/tr_001_006. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_007. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_008. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_009. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_010. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_011. It exists already.
